In [14]:
# Load modules
import datacube
import os
import sys
import warnings
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.image as mpimg
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
import geopandas as gp
import fiona
from datacube import helpers

import rasterio
import sklearn
import graphviz 
import pdb
from rasterio.features import rasterize

# Import external functions from dea-notebooks using relative link to 10_Scripts
sys.path.append('/g/data/u46/users/sc0554/dea-notebooks/Scripts')
from dea_bandindices import calculate_indices
import dea_classificationtools
from dea_plotting import display_map

In [15]:
#'Wrappers' to translate xarrays to np arrays and back for interfacing with sklearn models
def sklearn_flatten(input_xr):
    """
    Reshape a DataArray or Dataset with spatial (and optionally temporal) structure into 
    an np.array with the spatial and temporal dimensions flattened into one dimension.
    
    This flattening procedure enables DataArrays and Datasets to be used to train and predict
    with sklearn models.
    
    Last modified: September 2019
    
    Parameters
    ----------  
        input_xr : a DataArray or Dataset. Must have dimensions 'x' and 'y', may have dimension 'time'.
                   Dimensions other than 'x', 'y' and 'time' are unaffected by the flattening.
                   
    Returns
    ----------
        input_np : a numpy array corresponding to input_xr.data (or input_xr.to_array().data), with
                   dimensions 'x','y' and 'time' flattened into a single dimension, which is the first
                   axis of the returned array. input_np contains no NaNs.
    
    """
#     pdb.set_trace()
    #cast input Datasets to DataArray
    if isinstance(input_xr,xr.Dataset):
        input_xr = input_xr.to_array()
    
    #stack across pixel dimensions, handling timeseries if necessary
    if 'time' in input_xr.dims:
        stacked = input_xr.stack(z=['x','y','time'])
    else:
        stacked = input_xr.stack(z=['x','y'])
        
    #finding 'bands' dimensions in each pixel - these will not be flattened as their context is important for sklearn
    pxdims = []
    for dim in stacked.dims:
        if dim != 'z':
            pxdims.append(dim)
    
    #mask NaNs - we mask pixels with NaNs in *any* band, because sklearn cannot accept NaNs as input
    mask = np.isnan(stacked)
    if len(pxdims)!=0:
        mask = mask.any(dim=pxdims)
        
    #turn the mask into a numpy array (boolean indexing with xarrays acts weird)
    mask=mask.data
    #the dimension we are masking along ('z') needs to be the first dimension in the underlying np array for
    #the boolean indexing to work
    stacked = stacked.transpose('z',*pxdims)
    input_np = stacked.data[~mask]
    
    return input_np

In [16]:
dc = datacube.Datacube(app = 'classifiers')

In [5]:
#open the shapefile containing labelled data
# shp_path='/g/data1a/u46/users/sc0554/LCCS/LCCS/decision_tree/training_data/training_samples_2015_-11_-35.shp'
# shp_path='/g/data1a/u46/users/sc0554/LCCS/LCCS/decision_tree/training_data/LANDSCAPE_SALandCover_TrainingData_PointsConsolidated_SAOnly.shp'
shp_path = '/g/data1a/r78/LCCS_Aberystwyth/training_data/2015/Cell_-13_-36_2015.shp'
shapes=fiona.open(shp_path,'r')
# shp = gp.read_file(shp_path)
crs=geometry.CRS(shapes.crs_wkt)
# field = 'Classvalue'
# field = 'CLASS_VALU'
field = 'classnum'
product = 'ls8_nbart_geomedian_annual'
query = {
         'time': ('2015-01-01', '2015-02-01')
         }

In [11]:
for feature in shapes[0:1]:
    print(feature['geometry'])

{'type': 'Polygon', 'coordinates': [[(-1292600.0, -3500250.0), (-1292600.0, -3500200.0), (-1292625.0, -3500200.0), (-1292625.0, -3500275.0), (-1292675.0, -3500275.0), (-1292675.0, -3500175.0), (-1292650.0, -3500175.0), (-1292650.0, -3500150.0), (-1292625.0, -3500150.0), (-1292625.0, -3500125.0), (-1292525.0, -3500125.0), (-1292525.0, -3500200.0), (-1292500.0, -3500200.0), (-1292500.0, -3500225.0), (-1292525.0, -3500225.0), (-1292525.0, -3500250.0), (-1292600.0, -3500250.0)]]}


In [13]:
type(feature)

dict

In [6]:
bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']

In [7]:
# affine of single pixel is nans fyi

feature_rast_list = []
for feature in shapes[0]:
        
        # Datacube load the data near each feature in a shapefile, this returns a square / rectamg;e
        f_geometry=feature['geometry']
        geom=geometry.Geometry(f_geometry,crs=crs)
        query['geopolygon'] = geom
        data = dc.load(product=product, group_by='solar_day', **query)
        print(feature['geometry]'])
        # Rasterise the feature
        mask = rasterize([(feature['geometry'], feature['properties'][field])],
                                           out_shape=data.isel(time=0).blue.shape,
                                           transform=data.affine
                                          )
        
        mask = xr.DataArray(mask, coords=(data.y, data.x))

        # Mask the data so area outside the shapefile is NaN
        data = data.where(mask == feature['properties'][field], np.nan)
        
        # Calculate indices
        data = calculate_indices(data, 'BUI', collection='ga_ls_2')
        data = calculate_indices(data, 'BSI', collection='ga_ls_2')
        data = calculate_indices(data, 'BSI', collection='ga_ls_2')
        data = calculate_indices(data, 'NBI', collection='ga_ls_2')
        data = calculate_indices(data, 'EVI', collection='ga_ls_2')
        data = calculate_indices(data, 'NDWI', collection='ga_ls_2')
        data = calculate_indices(data, 'MSAVI', collection='ga_ls_2')
        
        # Remove time step if present
        data = data.isel(time=0)
        
        # Extract the label
        label = feature['properties'][field]

        # Append training data and label to list
        feature_rast_list.append((data, label))

TypeError: string indices must be integers

In [ ]:
# This plot should look like an the first feature of the training data with masking around it

feature_rast_list[0][0].blue.plot()

In [ ]:
feature_names = list(data.data_vars)
print(feature_names)
target_names = np.array(('Natural Terrestrial Vegetated', 'Artificial Surface', 'Natural Surface', 'Artificail Water', 'Natural Water'))
print(target_names)

In [ ]:
# Flatten arrays and append to list
flat_train_list = []
flat_val_list = []

for feature in feature_rast_list:
    # Flatten
    flat_train = sklearn_flatten(feature[0])
    
    # Make a list of labels for the same length as the training data
    flat_val = np.repeat(feature[1],flat_train.shape[0])
    flat_train_list.append(flat_train)
    flat_val_list.append(flat_val)

In [ ]:
# Stack list of arrays into single array
val_input = np.hstack(flat_val_list)
train_input = np.vstack(flat_train_list)
print(train_input.shape, val_input.shape)

# Model training

In [17]:
model_input = np.loadtxt('train_input.txt', delimiter = " ", skiprows=1)

In [19]:
model_input.shape

(5911666, 13)

In [ ]:
from sklearn import tree
# Initialise classifier
model = tree.DecisionTreeClassifier(random_state=0, max_depth=5)
# Fit classifier add "==215" to make a single class prediction.
model = model.fit(model_input[:,1:], model_input[:,0])

In [20]:
### RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
# Initialise classifier
model = RandomForestClassifier(n_estimators=100, verbose=2, n_jobs=-1)
# Fit classifier add "==215" to make a single class prediction.
model = model.fit(model_input[:,1:], model_input[:,0])

KeyboardInterrupt: 

In [ ]:
### DECISION TREE

from sklearn import tree
# Initialise classifier
model = tree.DecisionTreeClassifier(random_state=0, max_depth=3)
# Fit classifier add "==215" to make a single class prediction.
model = model.fit(train_input, val_input)

In [ ]:
### NEURAL NETWORK

from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1, activation='relu')
model = model.fit(train_input, val_input)

In [ ]:
# Prints the feature importance
print(dict(zip(feature_names, dtree.feature_importances_)))

In [ ]:
# Plots the structure of the tree
plt.figure(figsize=(25,8))
sklearn.tree.plot_tree(dtree, feature_names=feature_names, class_names=target_names) 

# Prediction

In [ ]:
# load the area you want to predict land cover here

# Lake Eyre
# x = (550000, 600000)
# y = (-3000000, -2950000)
# x = (-1000000, -950000)
# y = (-3400000, -3350000)
x = (-1200000, -1299850)
y = (-3600000, -3500125)

# # Coorong
# x = (600000, 700000)
# y = (-3950000, -3850000)

query = {'time': ('2015-01-01', '2015-02-01')}
query['x'] = (x[0], x[1])
query['y'] = (y[0], y[1])
query['crs'] = 'EPSG:3577'

In [ ]:
display_map(x, y, crs="EPSG:3577")

In [ ]:
new_data = dc.load(product=product, group_by='solar_day', **query)

In [ ]:
new_data = calculate_indices(new_data, 'BUI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'BSI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'BSI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'NBI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'EVI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'NDWI', collection='ga_ls_2')
new_data = calculate_indices(new_data, 'MSAVI', collection='ga_ls_2')

In [ ]:
predicted = dea_classificationtools.predict_xr(model, new_data)

In [ ]:
out = predicted.isel(time=0).transpose()
out = out.to_dataset(name="LCCS_L3")
out.attrs['crs']=geometry.CRS(data.crs)

In [ ]:
helpers.write_geotiff('rforest_merged.tif', out)